# VIA Soundcloud API

In [20]:
import soundcloud
import json
import requests
output_loc = '/home/will/Documents/Projects/Pi-Music-Server/music_backend/endpoints/res/wills_likes.json'
user_id='79333503'

In [21]:
import os
from dotenv import load_dotenv
load_dotenv()
SOUNDCLOUD_KEY = os.getenv("SOUNDCLOUD_ID")
print(SOUNDCLOUD_KEY)

587aa2d384f7333a886010d5f52f302a


In [22]:
client = soundcloud.Client(client_id=SOUNDCLOUD_KEY)
wills_songs = []
page_size = 200

In [23]:
response = client.get(f'/users/{user_id}/favorites', limit=page_size, linked_partitioning=1).__dict__['obj']

while True:
        for song in response['collection']:
            wills_songs.append({
                'platform': 'soundcloud',
                'url': song["permalink_url"],
                'title': song["title"],
                'artist': song["user"]["username"],
                'artwork_url': song["artwork_url"],
            })
        if('next_href' not in response):
            break
        response = json.loads(requests.get(response['next_href']).content)

In [25]:
#for song_title in [song['title'] for song in wills_songs]:
#    print(song_title)

In [26]:
with open(output_loc, 'w') as f:
    json.dump(wills_songs, f)

# VIA Selenium Webscraping